In [201]:
import logging
import os
import re
import argparse
import multiprocessing
import glob
import boto3
import yaml
import pandas as pd
import json
import sqlalchemy
from sqlalchemy.orm import sessionmaker

import sys
sys.path.append("..")
import src.helpers.helpers as h

In [ ]:
def connect_s3(access_id, access_key):
    
    s3 = boto3.resource('s3',
                    aws_access_key_id=access_id,
                    aws_secret_access_key=access_key)
    
    return s3

In [209]:
def get_s3_file_names(s3, s3_bucket_path):
    """Get all file names in an s3 bucket 

    Args:
        s3_bucket_path (str): S3 path to bucket containing all files to list (Ex: `s3://jdc-nu`)

    Returns: List of all S3 file locations

    """

    # parse s3 path for bucket name and prefix
    regex = r"s3://([\w._-]+)"
    m = re.match(regex, s3_bucket_path)
    s3bucket_name = m.group(1) 

    # Get s3 bucket handle
    s3bucket = s3.Bucket(s3bucket_name)

    # Get all file names in the `s3bucket`
    files = []
    for object in s3bucket.objects.all():
        file = object.key
        files.append(file)

    return files

In [40]:
files = get_s3_file_names("s3://jdc-nu",os.environ["AWS_ACCESS_ID"],os.environ["AWS_ACCESS_KEY"])

In [ ]:
def set_kitties(file, s3, )

In [202]:
def load_kitty_json(file, s3, bucket = "jdc-nu"):
    
    obj = s3.Object(bucket,file)
    file_content = obj.get()['Body'].read().decode('utf-8')
    json_content = json.loads(file_content)
    
    return json_content

In [199]:
# test = [load_kitty_json(file, s3, bucket = "jdc-nu") for file in files]


KeyboardInterrupt



In [162]:
out = load_kitty_json(files[0], s3)

In [160]:
ind = 9999
kitty_json = out["kitties"][ind]

In [203]:
def parse_attributes(kitty_json):

    id = kitty_json["id"]
    name = kitty_json["name"]
    image = kitty_json["image_url_png"]
    generation = kitty_json["generation"]
    birthday = kitty_json["created_at"]
    color = kitty_json["color"]
    fancy = kitty_json["is_fancy"]
    fancy_type = kitty_json["fancy_type"]
    exclusive = kitty_json["is_exclusive"]
    cooldown = kitty_json["status"]["cooldown_index"]
    purrs = kitty_json["purrs"]["count"]
    watches = kitty_json["watchlist"]["count"]
    hatched = kitty_json["hatched"]
    prestige = kitty_json["is_prestige"]
    prestige_type = kitty_json["prestige_type"]
    prestige_ranking = kitty_json["prestige_ranking"]
    fancy_ranking = kitty_json["fancy_ranking"]

    if(len(kitty_json["enhanced_cattributes"]) == 0):

        body = None
        coloreyes = None
        eyes = None
        pattern = None
        mouth = None
        colorprimary = None
        colorsecondary = None
        colortertiary = None

    else:

        body = kitty_json["enhanced_cattributes"][0]["description"] #body
        coloreyes = kitty_json["enhanced_cattributes"][1]["description"] #coloreyes
        eyes = kitty_json["enhanced_cattributes"][2]["description"] #eyes
        pattern = kitty_json["enhanced_cattributes"][3]["description"] #pattern
        mouth = kitty_json["enhanced_cattributes"][4]["description"] #mouth
        colorprimary = kitty_json["enhanced_cattributes"][5]["description"] #colorprimary
        colorsecondary = kitty_json["enhanced_cattributes"][6]["description"] #colorsecondary
        colortertiary = kitty_json["enhanced_cattributes"][7]["description"] #colortertiary
    
    if(len(kitty_json["matron"]) == 0):
        
        mother_id = None
        mother_fancy = None
        mother_exclusive = None
        
    else:
        
        mother_id = kitty_json["matron"]["id"]
        mother_fancy = kitty_json["matron"]["is_fancy"]
        mother_exclusive = kitty_json["matron"]["is_exclusive"]
    
    
    if(len(kitty_json["sire"]) == 0):
        
        father_id = None
        father_fancy = None
        father_exclusive = None
        
    else:
        
        father_id = kitty_json["sire"]["id"]
        father_fancy = kitty_json["sire"]["is_fancy"]
        father_exclusive = kitty_json["sire"]["is_exclusive"]
    
    
    if(len(kitty_json["auction"]) == 0):
        
        start_price = None
        end_price = None
        current_price = None
        auction_type = None
        auction_start = None
        auction_end = None
        auction_duration = None
        
    else:
        
        start_price = kitty_json["auction"]["start_price"]
        end_price = kitty_json["auction"]["end_price"]
        current_price = kitty_json["auction"]["current_price"]
        auction_type = kitty_json["auction"]["type"]
        auction_start = kitty_json["auction"]["start_time"]
        auction_end = kitty_json["auction"]["end_time"]
        auction_duration = kitty_json["auction"]["duration"]
    
    return [id, name, image, generation, birthday, color, fancy,
    fancy_type, exclusive, cooldown, purrs, watches, hatched, 
    prestige, prestige_ranking, fancy_ranking, body, mouth, eyes,
    pattern, colorprimary, colorsecondary, colortertiary, coloreyes,
    mother_id, mother_fancy, mother_exclusive, father_id,
    father_fancy, father_exclusive, start_price, end_price,
    current_price, auction_type, auction_start, auction_end,
    auction_duration]

In [204]:
def kitties_to_sql(kitties_json, engine_string):
    
    kitty_list = [parse_attributes(kitty) for kitty in kitties_json["kitties"]]
    
    kitties_df = pd.DataFrame(kitty_list)
    kitties_df.drop(0, axis=1)
    
    column_names = ["id", "name", "image", "generation", "birthday", "color", "fancy",
    "fancy_type", "exclusive", "cooldown", "purrs", "watches", "hatched", 
    "prestige", "prestige_ranking", "fancy_ranking", "body", "mouth", "eyes",
    "pattern", "colorprimary", "colorsecondary", "colortertiary", "coloreyes",
    "mother_id", "mother_fancy", "mother_exclusive", "father_id",
    "father_fancy", "father_exclusive", "start_price", "end_price",
    "current_price", "auction_type", "auction_start", "auction_end",
    "auction_duration"]
    
    kitties_df.columns = column_names
    
    engine = create_connection(engine_string=engine_string)
    
    kitties_df.to_sql("kitties", engine, if_exists="append", index = False)
    
    return True

In [164]:
test = [parse_attributes(kitty) for kitty in out["kitties"]]

In [165]:
test

[[1,
  'Genesis',
  'https://img.cryptokitties.co/0x06012c8cf97bead5deae237070f9587f8e7a266d/1.png',
  0,
  '2017-11-23T06:19:59Z',
  'sizzurp',
  True,
  'Genesis',
  True,
  0,
  1587,
  0,
  True,
  False,
  None,
  1,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [2,
  'Riker | Founder Cat #2',
  'https://img.cryptokitties.co/0x06012c8cf97bead5deae237070f9587f8e7a266d/2.png',
  0,
  '2017-11-23T06:19:59Z',
  'chestnut',
  False,
  None,
  False,
  0,
  84,
  0,
  True,
  False,
  None,
  None,
  'himalayan',
  'soserious',
  'simple',
  'jaguar',
  'orangesoda',
  'royalpurple',
  'kittencream',
  'chestnut',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [3,
  'Founder Cat #3',
  'https://img.cryptokitties.co/0x06012c8cf97bead5deae237070f9587f8e7a266d/3.png',
  0,
  '2017-11-23T06:19:59Z',
  'strawber

In [188]:
test_df = pd.DataFrame(test)

In [189]:
test_df.drop(0, axis=1)

,1,2,3,4,5,6,7,8,9,10,...,27,28,29,30,31,32,33,34,35,36
0,Genesis,https://img.cryptokitties.co/0x06012c8cf97bead...,0,2017-11-23T06:19:59Z,sizzurp,True,Genesis,True,0,1587,...,NaN,None,None,None,None,None,None,None,None,None
1,Riker | Founder Cat #2,https://img.cryptokitties.co/0x06012c8cf97bead...,0,2017-11-23T06:19:59Z,chestnut,False,None,False,0,84,...,NaN,None,None,None,None,None,None,None,None,None
2,Founder Cat #3,https://img.cryptokitties.co/0x06012c8cf97bead...,0,2017-11-23T06:19:59Z,strawberry,False,None,False,0,73,...,NaN,None,None,None,None,None,None,None,None,None
3,Fluffy Founder,https://img.cryptokitties.co/0x06012c8cf97bead...,0,2017-11-23T06:19:59Z,chestnut,False,None,False,2,33,...,NaN,None,None,None,None,None,None,None,None,None
4,C-A-T-S Cats! Cats! Cats! - #5,https://img.cryptokitties.co/0x06012c8cf97bead...,0,2017-11-23T06:19:59Z,chestnut,False,None,False,0,72,...,NaN,None,None,600000000000000000000,600000000000000000000,600000000000000000000,sale,2018-03-25 14:30:59+00,2072-12-26 14:30:59+00,1728000000000
5,æ¶¦é¾™ğŸ�±â€�ğŸ‘“åˆ›ä¸–çŒ«ç�‹6#,https://img.cryptokitties.co/0x06012c8cf97bead...,0,2017-11-23T06:19:59Z,mintgreen,False,None,False,0,25,...,NaN,None,None,None,None,None,None,None,None,None
6,Lucky,https://img.cryptokitties.co/0x06012c8cf97bead...,0,2017-11-23T06:19:59Z,topaz,False,None,False,0,23,...,NaN,None,None,None,None,None,None,None,None,None
7,Founder Cat #8,https://img.cryptokitties.co/0x06012c8cf97bead...,0,2017-11-23T06:20:08Z,sizzurp,False,None,False,7,66,...,NaN,None,None,888000000000000000000,888000000000000000000,888000000000000000000,sale,2017-12-17 17:49:04+00,2020-05-23 17:49:04+00,76723200000
8,æ¶¦é¾™ğŸ�±â€�ğŸ‘“çŒ«ç�‹9#,https://img.cryptokitties.co/0x06012c8cf97bead...,0,2017-11-23T06:20:35Z,mintgreen,False,None,False,0,25,...,NaN,None,None,None,None,None,None,None,None,None
9,Founder Cat #10,https://img.cryptokitties.co/0x06012c8cf97bead...,0,2017-11-23T06:20:44Z,chestnut,False,None,False,0,53,...,NaN,None,None,None,None,None,None,None,None,None


In [192]:
column_names = ["id", "name", "image", "generation", "birthday", "color", "fancy",
    "fancy_type", "exclusive", "cooldown", "purrs", "watches", "hatched", 
    "prestige", "prestige_ranking", "fancy_ranking", "body", "mouth", "eyes",
    "pattern", "colorprimary", "colorsecondary", "colortertiary", "coloreyes",
    "mother_id", "mother_fancy", "mother_exclusive", "father_id",
    "father_fancy", "father_exclusive", "start_price", "end_price",
    "current_price", "auction_type", "auction_start", "auction_end",
    "auction_duration"]

In [193]:
test_df.columns = column_names

In [205]:
def create_connection(host='127.0.0.1', database="", sqltype="mysql+pymysql", port=3308,
                      user_env="amazonRDS_user", password_env="amazonRDS_pw",
                      username=None, password=None, dbconfig=None, engine_string=None):

    if engine_string is None:
        if dbconfig is not None:
            with open(dbconfig, "r") as f:
                db = yaml.load(f)

            host = db["host"]
            database = ifin("dbname", db, "")
            sqltype = ifin("type", db, sqltype)
            port = db["port"]
            user_env = db["user_env"]
            password_env = db["password_env"]

        username = os.environ.get(user_env) if username is None else username
        password = os.environ.get(password_env) if password is None else password

        engine_string = "{sqltype}://{username}:{password}@{host}:{port}/{database}"
        engine_string = engine_string.format(sqltype=sqltype, username=username,
                                             password=password, host=host, port=port, database=database)

    conn = sqlalchemy.create_engine(engine_string)

    return conn

def get_session(engine=None, engine_string=None):
    """

    Args:
        engine_string: SQLAlchemy connection string in the form of:

            "{sqltype}://{username}:{password}@{host}:{port}/{database}"

    Returns:
        SQLAlchemy session
    """

    if engine is None and engine_string is None:
        return ValueError("`engine` or `engine_string` must be provided")
    elif engine is None:
        engine = create_connection(engine_string=engine_string)

    Session = sessionmaker(bind=engine)
    session = Session()

    return session

engine_string='sqlite:///../data/kitties.db'
engine = create_connection(engine_string=engine_string)
session = get_session(engine_string=engine_string)

In [197]:
test_df.to_sql("kitties", engine, if_exists="append", index = False)

In [211]:
def land_kitties(engine_string, bucket = "jdc-nu", access_id = os.environ["AWS_ACCESS_ID"], access_key = os.environ["AWS_ACCESS_KEY"]):

    s3 = boto3.resource('s3',
                    aws_access_key_id=access_id,
                    aws_secret_access_key=access_key)

    files = get_s3_file_names(s3, "s3://jdc-nu")

    cntr = 0
    for file in files:

        kitties_json = load_kitty_json(file, s3, bucket)
        kitties_to_sql(kitties_json, engine_string)

        cntr = cntr + len(kitties_json["kitties"])
        print(cntr)

    return

In [206]:
engine_string='sqlite:///../data/kitties.db'
bucket = "jdc-nu"
access_id = os.environ["AWS_ACCESS_ID"]
access_key = os.environ["AWS_ACCESS_KEY"]

In [212]:
land_kitties(engine_string, bucket, access_id, access_key)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000


KeyboardInterrupt: 

In [529]:
engine_string='sqlite:///../data/kitties.db'
engine = create_connection(engine_string=engine_string)

In [530]:
kitty_data = pd.read_sql("select * from kitties", engine)

In [577]:
training = kitty_data[kitty_data.auction_type == "sale"]
#validation = kitty_data[kitty_data.auction_type != "sale"]

In [578]:
training.loc[:,["current_price"]] = training.loc[:,["current_price"]]*.0000000000001

In [579]:
training["current_price"]

4         6.000000e+07
7         8.880000e+07
11        8.000000e+08
15        1.490000e+07
16        1.770000e+07
17        3.000000e+07
19        9.600000e+06
25        9.600000e+06
26        9.900000e+06
27        2.500000e+07
34        1.000000e+07
37        1.300000e+07
42        2.280000e+07
54        1.100000e+07
58        8.900000e+06
66        9.800000e+06
80        2.688595e+06
81        1.000000e+08
84        2.990000e+06
86        1.688000e+08
90        3.500000e+06
98        1.000000e+07
106       5.000000e+06
107       2.351135e+06
109       5.000000e+06
112       5.000000e+06
114       2.996566e+07
131       2.000000e+06
145       5.000000e+06
265       1.000000e+07
              ...     
139851    5.000000e+03
139852    6.000000e+03
139855    1.000000e+03
139857    1.850000e+05
139868    2.000000e+03
139870    7.000000e+02
139874    1.000000e+03
139881    4.440000e+04
139882    5.000000e+02
139884    5.000000e+02
139887    5.000000e+02
139889    5.350000e+05
139892    4

In [580]:
# to be set in yaml
full = ["generation", "color", "fancy","fancy_type", "exclusive", 
            "cooldown", "purrs", "watches", "prestige", 
            "prestige_ranking", "fancy_ranking", "body", "mouth", 
            "eyes", "pattern", "colorprimary", "colorsecondary", 
            "colortertiary", "coloreyes", "current_price"]
# features_plus = ["generation", "color", "fancy","fancy_type", "exclusive", 
#             "cooldown", "purrs", "watches", "prestige", 
#             "prestige_ranking", "fancy_ranking", "body", "mouth", 
#             "eyes", "pattern", "colorprimary", "colorsecondary", 
#             "colortertiary", "coloreyes"]
features = ["generation", "fancy", "exclusive", "cooldown", "purrs", "watches", "prestige"]
# categoricals = ["color", "fancy_type", "fancy_ranking", "prestige_ranking", "body", 
#                 "mouth", "eyes", "pattern", "colorprimary", "colorsecondary", 
#                 "colortertiary", "coloreyes"]
target = ["current_price"]

In [581]:
#train = training.filter(items=full)
X = training.filter(items=feats)
y = training.filter(items=target)


In [582]:
y

,current_price
4,6.000000e+07
7,8.880000e+07
11,8.000000e+08
15,1.490000e+07
16,1.770000e+07
17,3.000000e+07
19,9.600000e+06
25,9.600000e+06
26,9.900000e+06
27,2.500000e+07


In [583]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import ensemble
# from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
# from sklearn import preprocessing 


In [584]:
random_seed = 11
split = 0.9

In [585]:
X, y = shuffle(X, y, random_state=random_seed)
offset = int(X.shape[0] * split)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

In [593]:
params = {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)

In [594]:
clf.fit(X_train, y_train)

c:\users\jd cook\anaconda3\envs\avc\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.01, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [595]:
mae = mean_absolute_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mae)

MSE: 2431656211739.3247


In [608]:
import pickle
with open('../models/kitties_model.pkl', 'wb') as file:
    pickle.dump(clf, file)

In [490]:
with open(f'../models/kitties_model.pkl', 'rb') as f:
    clf = pickle.load(f)

In [605]:
pred = pd.read_sql("select * from kitties where id = " + str(3), engine)

In [606]:
X_pred = pred.filter(items=feats)

In [607]:
clf.predict(X_pred)

array([2.05681512e+12])

In [609]:
img = pd.read_sql("select image from kitties where id = " + str(3), engine)

In [612]:
img["image"][0]

'https://img.cryptokitties.co/0x06012c8cf97bead5deae237070f9587f8e7a266d/3.png'